In [21]:
#imports
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas
from sympy import init_printing
from IPython.display import display
from numpy import genfromtxt
from mpmath import diff
import time
import functools
init_printing(use_latex='mathjax')

### Загрузка данных

In [22]:
data = pandas.read_csv('train.csv', sep=',').as_matrix()
samples = data[:35000, 1:]
prepared_samples = np.hstack((samples, np.ones((samples.shape[0], 1))))
labels = data[:35000, 0]

test_samples = data[35000:, 1:]
prepared_test_samples = np.hstack((test_samples, np.ones((test_samples.shape[0], 1))))
test_labels = data[35000:, 0]

### 1. Многоклассовая классификация

#### Вычислим градиент функции $Q = -\frac{1}{\mathcal{l}}\sum_y\sum_i [y = i] \cdot \ln(p_i(W))$:
Сначала посчитаем градиент для одного семпла $x$ и для одного класса $W_i$:

$Q(W) = -[y = i]\cdot\log(p_i(W)) = -[y = i]\cdot\log\frac{exp(W_{i}x)}{\sum_{j}exp(W_{j}x)} = [y = i]\cdot(\log(\sum_{j}exp(W_{j}x)) - \log(exp(W_{i}x))) = [y = i]\cdot(\log(\sum_{j}exp(W_{j}x)) - W_{i}x)$

Отсюда $Q'(W_i^k) = [y = i]\cdot(\log(\sum_{j}exp(W_{j}x)) - W_{i}x)' = [y = i]\cdot(\frac{\sum_{j}exp(W_{j}x)'}{\sum_{j}exp(W_{j}x)} - x_k) = [y = i]\cdot(\frac{x_k\cdot\sum_{j}exp(W_{j}x)'}{\sum_{j}exp(W_{j}x)} - x_k) = $

$ = x_k\cdot[y = i]\cdot(\frac{\sum_{j}exp(W_{j}x)'}{\sum_{j}exp(W_{j}x)} - 1)$

In [ ]:
def gradient():
    pass

def loss_function():
    pass



def get_subsets(sample_set, label_set, subset_size=1):
    perm = np.random.permutation(len(sample_set))
    shuffled_samples = sample_set[perm]
    shuffled_labels = label_set[perm]
    int_bound = len(shuffled_samples) // subset_size
    for i in range(int_bound):
        yield ((shuffled_samples[i*subset_size:(i+1)*subset_size],
                        shuffled_labels[i*subset_size:(i+1)*subset_size]))

#### Adam:

In [ ]:
def adam(samples, labels, gradient_function, delta, delta_coef, delta_coef_degree,
         G, G_coef, G_coef_degree, start, step=0.01):
    EPSILON = 1e-10
    current_location = start.copy()
    gradient = gradient_function(samples, labels, start)
    delta = delta_coef * delta + (1 - delta_coef) * gradient
    G = G_coef * G + (1 - G_coef) * gradient**2
    current_location -= (step / (1 - delta_coef_degree) / (1 - G_coef_degree) * delta / (np.sqrt(G) + EPSILON))[0]
    delta_coef_degree *= delta_coef
    G_coef_degree *= G_coef
    return current_location

delta_coef = 0.1
G_coef = 0.95
size = 600
current_weights = np.zeros((10, prepared_samples.shape[1]))
G = np.zeros_like(current_weights)
delta = 0
delta_coef_degree = delta_coef
G_coef_degree = G_coef
steps_number = 0
while True:
    subsets = get_subsets(prepared_samples, labels, size)
    old_weights = current_weights.copy()
    for subset in subsets:
        current_weights = adam(subset[0], subset[1], gradient, delta, delta_coef, delta_coef_degree,
                           G, G_coef, G_coef_degree, current_weights, step=1e-4)
    steps_number += 1
    if abs(loss_function(current_weights) - loss_function(old_weights) < 1e-3:
        break

### 2. Полносвязанные нейронные сети

**Задание**
1. Предложите значения $w$ и $b$, чтобы $y$ реализовала операторы *and*, *or*, *not*.
Для начала условимся, что функция Хэвисайда задаётся, как равная единице при $arg \geq 0$ и равная нулю при $arg < 0$

Тогда:

*AND*: $\theta(w_{1}x_1 + w_{2}x_2 + b) = \theta(x_1 + x_2 - 2)$, т.е. $w_1 = w_2 = 1, b = -2 $

*OR*: $\theta(x_1 + x_2 - 1)$, т.е. $w_1 = w_2 = 1, b = -1 $

*NOT($x_1$)*: $\theta(-x_1)$, т.е. $w_1 = -1, w_2 = b = 0 $
2. Приведите пример булевой функции, которая не может быть представлена в виде $y$?

     $XOR(0, 0) = 0 \Rightarrow \theta(0w_1 + 0w_2 + b) = \theta(b) = 0 \Rightarrow b < 0$

     $XOR(0, 1) = 1 \Rightarrow \theta(w_2 + b) = 1 \Rightarrow w_2 \geq -b$
     
     $XOR(1, 0) = 1 \Rightarrow \theta(w_1 + b) = 1 \Rightarrow w_1 \geq -b$
     
     $w_1 \geq -b, w_2 \geq -b \Rightarrow w_1 + w_2 \geq -2b > -b$
     
     $XOR(1, 1) = 0 \Rightarrow \theta(w_1 + w_2 + b) = 0 \Rightarrow w_1 + w_2 < -b$, противоречие

Пусть теперь $x \in \{0, 1\}^{n}$, а $y = \theta(W_2 \cdot \theta(W_1x + b_1) + b_2)$

Аналогично обычному *AND*, $x \wedge \neg y = \theta(x - y - 2)$, $\neg x \wedge y = \theta(y - x - 2)$

Тогда если представить $F = x_1 \oplus x_2$ как $x_1 \wedge \neg x_2 \vee \neg x_1 \wedge x_2$, то $F = \theta(x_1 - x_2 - 2) \vee \theta(x_2 - x_1 - 2) = \theta(\theta_1(x_1 - x_2 - 2) + \theta_2(x_2 - x_1 - 2) - 1)$

Содержимое $\theta_1$ и $\theta_2$ получается, очевидно умножением соответствующих строк матрицы $W_1$ на вектор $x$ и прибавлением соответствующей компоненты вектора $b_1$, Содержимое внешнего $\theta$ тогда - сумма $b_2 = -1$ и скалярного произведения вектора $(\theta_1, \theta_2)$ на вектор $W_2 = (1, 1)$. Понятно, что это и есть вычисление функции $y$, таким образом, функция выразима с помощью данных средств.

Также понятно, что таким образом мы можем выразить ДНФ или КНФ, т.к. функции *AND* и *OR* без каких-либо проблем расширяются для произвольного количества переменных (положительного или отрицательного - $\neg$ - знака), которыми выражается любая логическая формула